# The goal of this document is to:

### Merge bridge and socioeconomic data

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 
from sklearn.metrics import accuracy_score
from random import randrange
import warnings
import math
from sklearn import tree
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import seaborn as sn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split #from imblearn import under_sampling, over_sampling

# Bridge Data with Demographic and socioeconomic variables 
#### 4. Retrieve relevant demographic and socioeconomic variables from the ACS 2020 5 year estimates. Map each census tract to the municipality they belong to. Aggregate these variables at municipality level and obtain summary statistics for each municipality. Mrge the summarized socioeconomic and demographic statistics to each bridge data point based on municipality
IMPORTANT NOTE: Census tracts must stay within a county and therefore a state. They do not necessarily coincide within any other geography. For example, although some census tracts follow place boundaries, there is no rule that says they must stay within a place. This explains why some same census tracts appear in different Municipalities
https://www.census.gov/newsroom/blogs/random-samplings/2014/07/understanding-geographic-relationships-counties-places-tracts-and-more.html<br>
Some municipalities will have nans, as there are not enough sample points in the census tracts.

In [2]:
#Read the excel file that contains the Census Tracts and the Municipality they belong to.
#Notice that Census Tracts in decimal format, e.g., XXXX.XX will be treated separately from Census Tracts in integer format
CensusTractdecimals_2020_Municipality = pd.read_excel("CensusTracts2020_Municipalities_AlleghenyCounty.xlsx", 
                                                      sheet_name = "Decimals")
CensusTractintegers_2020_Municipality = pd.read_excel("CensusTracts2020_Municipalities_AlleghenyCounty.xlsx", 
                                                      sheet_name = "Integers")
#Convert the census tract columns to string characters
CensusTractdecimals_2020_Municipality["CensusTract"] = CensusTractdecimals_2020_Municipality["CensusTract"].astype(str)
CensusTractintegers_2020_Municipality["CensusTract"] = CensusTractintegers_2020_Municipality["CensusTract"].astype(str)

#Create a dictionary, in which the keys are the census tracts and the values are the municipalities they belong to
CensusTracts_to_Municipality_dict = dict(zip(CensusTractintegers_2020_Municipality.CensusTract, 
                                             CensusTractintegers_2020_Municipality.Municipality))
CensusTracts_to_Municipality_dict.update(dict(zip(CensusTractdecimals_2020_Municipality.CensusTract, 
                                                  CensusTractdecimals_2020_Municipality.Municipality)))
#Convert the keys of the dictionary to string characters
CensusTracts_to_Municipality_dict = {str(key): str(value) for key, value in CensusTracts_to_Municipality_dict.items()}

#### Race
#### Information was retrieved from ACS 2020 5-year estimates. This dataframe describes the population composition in terms of race for each census tract. Data will be aggregated at the municipality level. Notice that some census tracts may not contain data points. This is due to these reasons: 1) the number of sample cases is too small or 2) no sample observations available. In this case, nans are ignored when aggregating values at the municipality level. Value were aggregate by adding all census tracts statistics or averaging all census tracts statistics.  The dataframe at municipality level contains absolute estimates and percentages (with respect to the total population) of race composition. 

In [3]:
#Read the census data 
race_df = pd.read_excel("Total_Population_Race_2020.xlsx")

#Retrieve the census tract and save it as a string
race_df["GEO_ID_CensusTract"] = race_df.Geography.str[14:].astype(str).str.lstrip("0")

#Add decimal points
race_df["CensusTract(decimal format)"] = race_df["GEO_ID_CensusTract"].str[:-2]+'.'+race_df["GEO_ID_CensusTract"].str[-2:]

#Replace the .00 to ""
race_df["CensusTract(decimal format)"] = race_df["CensusTract(decimal format)"].astype(str)
race_df["CensusTract(decimal format)"] = race_df["CensusTract(decimal format)"].str.replace(".00", "", regex=False)

#Create a new column that indicates the Municipality to which each row of census data frame belongs to
race_df['Municipality'] = race_df["CensusTract(decimal format)"].map(CensusTracts_to_Municipality_dict)

#The following df contains the number of inhabitants in each municipality according to their race
Municipality_race_df = pd.pivot_table(race_df, values=['Total population', 
                                                    "Hispanic or Latino (of any race)", 
                                                    "White alone", 
                                                    "Black or African American alone", 
                                                    "American Indian and Alaska Native alone", 
                                                    "Asian alone", 
                                                    "Native Hawaiian and Other Pacific Islander alone",
                                                    "Some other race alone", 
                                                    "Two or more races"], 
                                   index=['Municipality'],
                                   aggfunc={'Total population': np.sum,
                                            "Hispanic or Latino (of any race)" : np.sum, 
                                             "White alone" : np.sum, 
                                             "Black or African American alone": np.sum, 
                                             "American Indian and Alaska Native alone": np.sum, 
                                             "Asian alone": np.sum, 
                                             "Native Hawaiian and Other Pacific Islander alone": np.sum,
                                             "Some other race alone": np.sum, 
                                             "Two or more races": np.sum})

#find percentages
Municipality_race_df["%Hispanic"] = (Municipality_race_df["Hispanic or Latino (of any race)"] / 
                                     Municipality_race_df["Total population"]) * 100
Municipality_race_df["%White alone"] = (Municipality_race_df["White alone"] / 
                                        Municipality_race_df["Total population"]) * 100
Municipality_race_df["%Black or African American alone"] = (Municipality_race_df["Black or African American alone"] / 
                                                            Municipality_race_df["Total population"]) * 100
Municipality_race_df["%American Indian and Alaska Native alone"] = (
    Municipality_race_df["American Indian and Alaska Native alone"] / Municipality_race_df["Total population"]) * 100
Municipality_race_df["%Asian alone"] = (Municipality_race_df["Asian alone"] / 
                                        Municipality_race_df["Total population"]) * 100
Municipality_race_df["%Native Hawaiian and Other Pacific Islander alone"] = (
    Municipality_race_df["Native Hawaiian and Other Pacific Islander alone"] / 
    Municipality_race_df["Total population"]) * 100
Municipality_race_df["%Some other race alone"] = (Municipality_race_df["Some other race alone"] /
                                                  Municipality_race_df["Total population"]) * 100
Municipality_race_df["%Two or more races"] = (Municipality_race_df["Two or more races"] / 
                                              Municipality_race_df["Total population"]) * 100


#### Economic characteristics
#### Information was retrieved from ACS 2020 5-year estimates. This dataframe provides information regarding the total number of civilian labor force, unemployment rate among civilian labor force, mean household income, total number of workers, the number of workers according to their of means of transportation to work. Data will be aggregated at the municipality level. Notice that some census tracts may not contain data points. This is due to these reasons: 1) the number of sample cases is too small or 2) no sample observations available. In this case, nans are ignored when aggregating values at the municipality level. Value were aggregate by adding all census tracts statistics or averaging all census tracts statistics.  The dataframe at municipality level contains absolute estimates and percentages (with respect to the total number of workers) 

In [4]:
#Read the census data 
econ_characs_df = pd.read_excel("Econ_Characs_2020_data.xlsx")

#Retrieve the census tract and save it as a string
econ_characs_df["GEO_ID_CensusTract"] = econ_characs_df.Geography.str[14:].astype(str).str.lstrip("0")

#Add decimal points
econ_characs_df["CensusTract(decimal format)"] = econ_characs_df["GEO_ID_CensusTract"].str[:-2]+'.'+econ_characs_df["GEO_ID_CensusTract"].str[-2:]

#Replace the .00 to ""
econ_characs_df["CensusTract(decimal format)"] = econ_characs_df["CensusTract(decimal format)"].astype(str)
econ_characs_df["CensusTract(decimal format)"] = econ_characs_df["CensusTract(decimal format)"].str.replace(".00", "", regex=False)

#Create a new column that indicates the Municipality to which each row of census data frame belongs to
econ_characs_df['Municipality'] = econ_characs_df["CensusTract(decimal format)"].map(CensusTracts_to_Municipality_dict)

#The following df contains the average unemployment rate, household income in each municipality
Municipality_economic_characs_df = pd.pivot_table(econ_characs_df, values=['Total civilian labor force',
                                                                          'Unemployment Rate among civilian labor force',
                                                                           'Mean household income (dollars)',
                                                                           'Total number workers 16 years and over',
                                                                           'Total workers that commute with car, truck, or van  drove alone',
                                                                           'Total workers that commute with car, truck, or van  carpooled',
                                                                           'Total workers that commute with public transportation (excluding taxicab)',
                                                                           'Total workers that walked',
                                                                           'Total workers that commute by other means',
                                                                            'Total workers that worked from home'], 
                                                                   index=['Municipality'],
                                                                   aggfunc={'Total civilian labor force': np.sum,
                                                                            'Unemployment Rate among civilian labor force' : np.mean,
                                                                            'Mean household income (dollars)' : np.mean, 
                                                                           'Total number workers 16 years and over': np.sum,
                                                                           'Total workers that commute with car, truck, or van  drove alone': np.sum,
                                                                           'Total workers that commute with car, truck, or van  carpooled': np.sum,
                                                                           'Total workers that commute with public transportation (excluding taxicab)': np.sum,
                                                                           'Total workers that walked': np.sum,
                                                                           'Total workers that commute by other means': np.sum,
                                                                            'Total workers that worked from home': np.sum})

#Find percentages
econ_characs_df["%Total workers that commute with car, truck, or van  drove alone"] = (econ_characs_df["Total workers that commute with car, truck, or van  drove alone"] / econ_characs_df["Total number workers 16 years and over"]) * 100
econ_characs_df["%Total workers that commute with car, truck, or van  carpooled"] = (econ_characs_df["Total workers that commute with car, truck, or van  carpooled"] / econ_characs_df["Total number workers 16 years and over"]) * 100
econ_characs_df["%Total workers that commute with public transportation (excluding taxicab)"] = (econ_characs_df["Total workers that commute with public transportation (excluding taxicab)"] / econ_characs_df["Total number workers 16 years and over"]) * 100
econ_characs_df["%Total workers that walked"] = (econ_characs_df["Total workers that walked"] / econ_characs_df["Total number workers 16 years and over"]) * 100
econ_characs_df["%Total workers that commute by other means"] = (econ_characs_df["Total workers that commute by other means"] / econ_characs_df["Total number workers 16 years and over"]) * 100
econ_characs_df["%Total workers that worked from home"] = (econ_characs_df["Total workers that worked from home"] / econ_characs_df["Total number workers 16 years and over"]) * 100

#### Poverty information
#### Information was retrieved from ACS 2020 5-year estimates. This dataframe provides information regarding the total number of inhabitants living in poverty, and the number of inhabitants living below the poverty status at each census tract. Data will be aggregated at the municipality level. Notice that some census tracts may not contain data points. This is due to these reasons: 1) the number of sample cases is too small or 2) no sample observations available. In this case, nans are ignored when aggregating values at the municipality level. Value were aggregate by adding all census tracts statistics or averaging all census tracts statistics.  The dataframe at municipality level contains absolute estimates and percentages (with respect to the total population living in poverty status) 

In [5]:
#Read the census data 
poverty_status_df = pd.read_excel("PovertyStatus_data_2020.xlsx")

#Retrieve the census tract and save it as a string
poverty_status_df["GEO_ID_CensusTract"] = poverty_status_df.Geography.str[14:].astype(str).str.lstrip("0")

#Add decimal points
poverty_status_df["CensusTract(decimal format)"] = poverty_status_df["GEO_ID_CensusTract"].str[:-2]+'.'+poverty_status_df["GEO_ID_CensusTract"].str[-2:]

#Replace the .00 to ""
poverty_status_df["CensusTract(decimal format)"] = poverty_status_df["CensusTract(decimal format)"].astype(str)
poverty_status_df["CensusTract(decimal format)"] = poverty_status_df["CensusTract(decimal format)"].str.replace(".00", "", regex=False)

#Create a new column that indicates the Municipality to which each row of census data frame belongs to
poverty_status_df['Municipality'] = poverty_status_df["CensusTract(decimal format)"].map(CensusTracts_to_Municipality_dict)
#The following df contains number living in poverty and the number of people living below the poverty level
Municipality_poverty_df = pd.pivot_table(poverty_status_df, values=['Total population for whom poverty status is determined',
                                                                    'Total population for whom poverty status is determined (Below poverty level)'], 
                                                                   index=['Municipality'],
                                                                   aggfunc={'Total population for whom poverty status is determined' : np.sum,
                                                                            'Total population for whom poverty status is determined (Below poverty level)': np.sum})

Municipality_poverty_df["%Total population for whom poverty status is determined (Below poverty level)"] = (
    Municipality_poverty_df["Total population for whom poverty status is determined (Below poverty level)"]/
    Municipality_poverty_df["Total population for whom poverty status is determined"])*100

#### Educational attainment information
#### Information was retrieved from ACS 2020 5-year estimates. This dataframe provides information regarding the total number of households, the total population older than 25 years, and the population (older than 25 years old) with less than 9th grade. Data will be aggregated at the municipality level. Notice that some census tracts may not contain data points. This is due to these reasons: 1) the number of sample cases is too small or 2) no sample observations available. In this case, nans are ignored when aggregating values at the municipality level. Value were aggregate by adding all census tracts statistics or averaging all census tracts statistics.  The dataframe at municipality level contains absolute estimates and percentages (with respect to the total population older than 25 years) 

In [6]:
#Read the census data 
educational_attainment_df = pd.read_excel("EducationalAttainment_2020_data.xlsx")

#Retrieve the census tract and save it as a string
educational_attainment_df["GEO_ID_CensusTract"] = educational_attainment_df.Geography.str[14:].astype(str).str.lstrip("0")

#Add decimal points
educational_attainment_df["CensusTract(decimal format)"] = educational_attainment_df["GEO_ID_CensusTract"].str[:-2]+'.'+educational_attainment_df["GEO_ID_CensusTract"].str[-2:]

#Replace the .00 to ""
educational_attainment_df["CensusTract(decimal format)"] = educational_attainment_df["CensusTract(decimal format)"].astype(str)
educational_attainment_df["CensusTract(decimal format)"] = educational_attainment_df["CensusTract(decimal format)"].str.replace(".00", "", regex=False)

#Create a new column that indicates the Municipality to which each row of census data frame belongs to
educational_attainment_df['Municipality'] = educational_attainment_df["CensusTract(decimal format)"].map(CensusTracts_to_Municipality_dict)
#The following df contains the number of households, population of 25 years and over, and population with less than 9th grade completed
Municipality_educational_attainment_df = pd.pivot_table(educational_attainment_df, values=['Total number households',
                                                                    'Total population 25 years and over',
                                                                    'Total population 25 years and over with less than 9th grade'], 
                                                                   index=['Municipality'],
                                                                   aggfunc={'Total number households' : np.sum,
                                                                            'Total population 25 years and over': np.sum,
                                                                           'Total population 25 years and over with less than 9th grade' : np.sum})


#Find percentages
Municipality_educational_attainment_df["%Total population 25 years and over with less than 9th grade"] = (Municipality_educational_attainment_df["Total population 25 years and over with less than 9th grade"]/Municipality_educational_attainment_df["Total population 25 years and over"])*100



#### Merge the socioeconomic and demographic statistics of municipalities to the bridge dataset

In [7]:
#Read the file that contains the PA22 - 22 csv file with the municipality of Allegheny county where every bridge belongs to
Bridges_df = pd.read_csv("df.csv")
#Join the 4 different df that contains demographic information about municipalities 

Bridges_Municipalities_Demographics_df = Bridges_df.merge(right = Municipality_educational_attainment_df, 
                                                       how = "left", 
                                                       left_on = "Municipality", 
                                                       right_on = "Municipality")

Bridges_Municipalities_Demographics_df = Bridges_Municipalities_Demographics_df.merge(right = Municipality_poverty_df, 
                                                       how = "left", 
                                                       left_on = "Municipality", 
                                                       right_on = "Municipality")

Bridges_Municipalities_Demographics_df = Bridges_Municipalities_Demographics_df.merge(right = Municipality_economic_characs_df, 
                                                       how = "left", 
                                                       left_on = "Municipality", 
                                                       right_on = "Municipality")

Bridges_Municipalities_Demographics_df = Bridges_Municipalities_Demographics_df.merge(right = Municipality_race_df, 
                                                       how = "left", 
                                                       left_on = "Municipality", 
                                                       right_on = "Municipality")

FileNotFoundError: [Errno 2] No such file or directory: 'df.csv'

In [ ]:
## This dataframe contains socioeconomic and demographic statistics and characteristics of each bridge. Lets merge this df
## with the df that has already been prepared
Bridges_Municipalities_Demographics_df = Bridges_Municipalities_Demographics_df[["Original_row_number","Municipality",
"Total number households",
"Total population 25 years and over",
"Total population 25 years and over with less than 9th grade",
"%Total population 25 years and over with less than 9th grade",
"Total population for whom poverty status is determined",
"Total population for whom poverty status is determined (Below poverty level)",
"%Total population for whom poverty status is determined (Below poverty level)",
"Mean household income (dollars)",
"Total civilian labor force",
"Total number workers 16 years and over",
"Total workers that commute by other means",
"Total workers that commute with car, truck, or van  carpooled",
"Total workers that commute with car, truck, or van  drove alone",
"Total workers that commute with public transportation (excluding taxicab)",
"Total workers that walked",
"Total workers that worked from home",
"Unemployment Rate among civilian labor force",
"American Indian and Alaska Native alone",
"Asian alone",
"Black or African American alone",
"Hispanic or Latino (of any race)",
"Native Hawaiian and Other Pacific Islander alone",
"Some other race alone",
"Total population",
"Two or more races",
"White alone",
"%Hispanic",
"%White alone",
"%Black or African American alone",
"%American Indian and Alaska Native alone",
"%Asian alone",
"%Native Hawaiian and Other Pacific Islander alone",
"%Some other race alone",
"%Two or more races"]]

df.reset_index(inplace=True)
df2 = df.merge(right = Bridges_Municipalities_Demographics_df,
             how = "inner", 
             left_on = "index", 
             right_on = "Original_row_number")

df2.describe()

## Normalization of the socioeconomic and demographic data

In [ ]:
df2['Structure_Kind_Aluminum'] #confirming dummy variabkes are there

In [ ]:
continous_var=df.columns[90:] #include only demographic and socio econmoic data
df_norm=df2.copy()
df_norm = normalize(df_norm, continous_var) #normalize the columns

In [ ]:

for var in list(df_norm.columns):   #Checks for any columns remaining that are not in the correct format (numerical)
    print(var, df_norm[var].dtypes) 


In [ ]:
#municipality name must be dropped for classification modelb because it is in object format
df_norm.drop(columns = 'Municipality', inplace = True)
#df_norm['Municipality']

In [ ]:

for var in list(df_norm.columns):   #Checks for any columns remaining that are not in the correct format (numerical)
    print(var, df_norm[var].dtypes) 


In [ ]:
df_norm.dropna(inplace=True)  #remove NAs 

In [ ]:
df.to_csv = ('df_norm.csv') #saving locally